In [1]:
import cv2
import numpy as np
import mediapipe as mp
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from comtypes import CLSCTX_ALL
import math
import screen_brightness_control as sbc
import pyttsx3
import pygame  # Import pygame for background music

# Initialize pygame mixer
pygame.mixer.init()
pygame.mixer.music.load("Saaheb.mp3")  # Replace with your music file
pygame.mixer.music.set_volume(0.5)  # Set volume (0.0 to 1.0)
pygame.mixer.music.play(-1)  # Play in a loop

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Initialize PyCaw for system volume control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)  # Use _iid_
volume = interface.QueryInterface(IAudioEndpointVolume)
vol_range = volume.GetVolumeRange()
min_vol, max_vol = vol_range[:2]

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Start capturing video
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    success, img = cap.read()
    if not success:
        break

    # Flip the image horizontally for a mirror effect
    img = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Process the image with MediaPipe Hands
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            lm_list = []
            h, w, _ = img.shape  # Get frame size

            # Extract hand landmarks
            for id, lm in enumerate(hand_landmarks.landmark):
                lm_list.append((int(lm.x * w), int(lm.y * h)))

            if len(lm_list) >= 8:  # Ensure thumb and index finger are detected
                x1, y1 = lm_list[4]  # Thumb tip
                x2, y2 = lm_list[8]  # Index finger tip

                # Draw circles and line between thumb and index finger
                cv2.circle(img, (x1, y1), 10, (255, 0, 0), -1)
                cv2.circle(img, (x2, y2), 10, (255, 0, 0), -1)
                cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 3)

                # Calculate distance between thumb and index finger
                length = math.hypot(x2 - x1, y2 - y1)

                # Volume Control
                vol = np.interp(length, [20, 200], [min_vol, max_vol])
                volume.SetMasterVolumeLevel(vol, None)

                # Brightness Control
                brightness = np.interp(length, [20, 200], [0, 100])
                sbc.set_brightness(int(brightness))

                # 🎵 Music Volume Control
                music_vol = np.interp(length, [20, 200], [0.0, 1.0])
                pygame.mixer.music.set_volume(music_vol)

                # Pinch-to-Mute (Volume)
                if length < 20:  # If fingers are very close (pinch gesture)
                    volume.SetMute(1, None)  # Mute the volume
                    cv2.putText(img, "Muted", (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                else:
                    volume.SetMute(0, None)  # Unmute the volume

                # 🎵 Pinch Gesture to Play/Pause Music
                if length < 20:  # If fingers are close, toggle play/pause
                    if pygame.mixer.music.get_busy():
                        pygame.mixer.music.pause()  # Pause if playing
                        cv2.putText(img, "Music Paused", (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    else:
                        pygame.mixer.music.unpause()  # Resume if paused
                        cv2.putText(img, "Music Playing", (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Display Volume and Brightness
                vol_percent = int(np.interp(vol, [min_vol, max_vol], [0, 100]))
                cv2.putText(img, f'Vol: {vol_percent}%', (40, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                cv2.putText(img, f'Bright: {int(brightness)}%', (40, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            if len(lm_list) >= 12:  # Ensure at least 12 landmarks exist
                x1, y1 = lm_list[8]   # Index finger tip
                x2, y2 = lm_list[12]  # Middle finger tip

                # ✋ Stop Music with Two Fingers Open
                if abs(x1 - x2) > 50:
                    pygame.mixer.music.stop()
                    cv2.putText(img, "Music Stopped", (40, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Draw hand landmarks
            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display the image
    cv2.imshow("Hand Gesture Control", img)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Stop the music before exiting
pygame.mixer.music.stop()

# Release resources
cap.release()
cv2.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
